In [23]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("olgabelitskaya/horse-breeds")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\trh00\.cache\kagglehub\datasets\olgabelitskaya\horse-breeds\versions\27


In [24]:
import os

root = r'C:\Users\trh00\.cache\kagglehub\datasets\olgabelitskaya\horse-breeds\versions\27'
images = os.listdir(root)
images[:5]

['01_001.png', '01_002.png', '01_003.png', '01_004.png', '01_005.png']

In [25]:
# split into training and test
import random
random.seed(4)
random.shuffle(images)
split = int(0.8 * len(images))
train_images = images[:split]
test_images = images[split:]

print("Number of training images:", len(train_images))
print("Number of test images:", len(test_images))

Number of training images: 536
Number of test images: 134


In [50]:
import torch
from torch.utils.data import Dataset
# create transform
from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

class HorseBreedDataset(Dataset):
    def __init__(self, root, images, transform_input=None, transform_target=None):
        self.root = root
        self.images = images
        self.transform_input = transform_input
        self.transform_target = transform_target

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(os.path.join(self.root, self.images[idx]))
        input_image = self.transform_input(image)
        target = int(images[idx].split('_')[0])
        target = target - 1
        target = torch.tensor(target)
        return input_image, target
    
train_dataset = HorseBreedDataset(root, train_images, transform_input=transform)
test_dataset = HorseBreedDataset(root, test_images, transform_input=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)



In [42]:
for input_image, target in train_loader:
    print(input_image.shape, target)
    break

torch.Size([32, 3, 256, 256]) tensor([4, 0, 4, 5, 6, 3, 1, 0, 6, 5, 5, 5, 3, 0, 5, 2, 5, 3, 5, 0, 2, 1, 2, 6,
        0, 1, 3, 3, 4, 4, 6, 1])


In [28]:
import torch.nn as nn
import torch

# Define the CNN Model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolutional Layers
        self.conv1 = nn.Conv2d(in_channels=4, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)

        # Activation
        self.relu = nn.ReLU()
        
        # Pooling Layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Fully Connected Layers
        self.fc1 = nn.Linear(65536, 128)
        self.fc2 = nn.Linear(128, 7)  # 10 output classes for CIFAR-10

    def forward(self, x):
        # original shape 1, 250, 90
        # Convolution + ReLU + Pooling
        x = self.pool(self.relu(self.conv1(x)))  #  16, 125, 45
        x = self.pool(self.relu(self.conv2(x)))  # 32, 62, 22 
        x = self.pool(self.relu(self.conv3(x)))  # 64, 31, 11 
        
        # Flatten
        x = x.view(x.size(0), -1)  # (Batch, 64*4*4)
        
        # Fully Connected Layers
        x = self.relu(self.fc1(x))
        x = self.fc2(x)  # Logits
        
        return x                # Although this is a classification problem, we didn't apply softmax. Do you know why?👀 (Hint: CrossEntropyLoss has something to do here👀)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNModel().to(device)


In [29]:
from tqdm import tqdm    # Shows progress bar

# 🔹 Training Loop
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()  # Set model to training mode
    total_loss = 0
    correct = 0
    total = 0

    for images, labels in tqdm(dataloader):
        images, labels = images.to(device), labels.to(device)

        
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss

        optimizer.zero_grad()  # Reset gradients
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

        total_loss += loss.item()

        # Track accuracy
        outputs = torch.softmax(outputs, dim=1)
        predictions = outputs.argmax(dim=1)  # Get class with highest probability
        correct += (predictions == labels).sum().item()
        total += labels.size(0)

    avg_loss = total_loss / len(dataloader)
    accuracy = 100 * correct / total  # Compute accuracy in percentage
    return avg_loss, accuracy

# 🔹 Validation Loop
def validate(model, dataloader, criterion, device):
    model.eval()  # Set model to evaluation mode
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)  # Forward pass
            loss = criterion(outputs, labels)  # Compute loss
            total_loss += loss.item()

            # Compute accuracy
            outputs = torch.softmax(outputs, dim=1)
            predictions = outputs.argmax(dim=1)  # Get predicted class
            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    avg_loss = total_loss / len(dataloader)
    accuracy = 100 * correct / total  # Compute accuracy in percentage
    return avg_loss, accuracy


In [30]:
import torch.optim as optim

# Initialize the model

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Multi-class Classification loss (Input: Logits, not probabilities)
optimizer = optim.AdamW(model.parameters(), lr=0.0001)  # Adam optimizer
num_epochs = 20 # Number of epochs


# Lists to store metrics
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

# Training process
for epoch in range(num_epochs):
    train_loss, train_accuracy = train_one_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_accuracy = validate(model, test_loader, criterion, device)

    # Store metrics
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracies.append(train_accuracy)
    val_accuracies.append(val_accuracy)

    print(f"Epoch {epoch+1}/{num_epochs}: "
          f"Train Loss={train_loss:.4f}, Train Accuracy={train_accuracy:.2f}%, "
          f"Val Loss={val_loss:.4f}, Val Accuracy={val_accuracy:.2f}%")


100%|██████████| 17/17 [00:06<00:00,  2.78it/s]


Epoch 1/20: Train Loss=1.8882, Train Accuracy=25.00%, Val Loss=1.9232, Val Accuracy=23.13%


100%|██████████| 17/17 [00:05<00:00,  3.02it/s]


Epoch 2/20: Train Loss=1.8289, Train Accuracy=30.78%, Val Loss=1.9218, Val Accuracy=22.39%


100%|██████████| 17/17 [00:07<00:00,  2.18it/s]


...

Epoch 18/20: Train Loss=0.4116, Train Accuracy=88.06%, Val Loss=3.9748, Val Accuracy=17.91%


100%|██████████| 17/17 [00:06<00:00,  2.70it/s]


Epoch 19/20: Train Loss=0.3614, Train Accuracy=90.49%, Val Loss=4.0626, Val Accuracy=17.91%


100%|██████████| 17/17 [00:07<00:00,  2.25it/s]


Epoch 20/20: Train Loss=0.2863, Train Accuracy=94.78%, Val Loss=4.3242, Val Accuracy=16.42%


## try it in a pretrained model

In [51]:
from torchvision.models import resnet18

model = resnet18(pretrained=True)
# freeze all layers
# for param in model.parameters():
#     param.requires_grad = False
model.conv1 = nn.Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = nn.Linear(512, 7)  # Change output layer to 7 classes

model = model.to(device)
model

c:\Users\trh00\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\trh00\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [52]:
import torch.optim as optim

# Initialize the model

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Multi-class Classification loss (Input: Logits, not probabilities)
optimizer = optim.AdamW(model.parameters(), lr=0.0001)  # Adam optimizer
num_epochs = 10 # Number of epochs


# Lists to store metrics
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

# Training process
for epoch in range(num_epochs):
    train_loss, train_accuracy = train_one_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_accuracy = validate(model, test_loader, criterion, device)

    # Store metrics
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracies.append(train_accuracy)
    val_accuracies.append(val_accuracy)

    print(f"Epoch {epoch+1}/{num_epochs}: "
          f"Train Loss={train_loss:.4f}, Train Accuracy={train_accuracy:.2f}%, "
          f"Val Loss={val_loss:.4f}, Val Accuracy={val_accuracy:.2f}%")


100%|██████████| 17/17 [00:08<00:00,  1.98it/s]


Epoch 1/10: Train Loss=1.6008, Train Accuracy=41.04%, Val Loss=2.0753, Val Accuracy=16.42%


100%|██████████| 17/17 [00:06<00:00,  2.48it/s]


Epoch 2/10: Train Loss=0.6478, Train Accuracy=80.97%, Val Loss=2.5482, Val Accuracy=15.67%


100%|██████████| 17/17 [00:06<00:00,  2.50it/s]


...

Epoch 8/10: Train Loss=0.0139, Train Accuracy=100.00%, Val Loss=3.5029, Val Accuracy=14.93%


100%|██████████| 17/17 [00:06<00:00,  2.56it/s]


Epoch 9/10: Train Loss=0.0121, Train Accuracy=100.00%, Val Loss=3.5608, Val Accuracy=15.67%


100%|██████████| 17/17 [00:06<00:00,  2.51it/s]


Epoch 10/10: Train Loss=0.0103, Train Accuracy=100.00%, Val Loss=3.5675, Val Accuracy=16.42%


In [53]:
test_loss, test_accuracy = validate(model, test_loader, criterion, device)
print(f"Test Loss={test_loss:.4f}, Test Accuracy={test_accuracy:.2f}%")

Test Loss=3.5675, Test Accuracy=16.42%
